### Task 3.a Calculating Cohen's Kappa

Importing Necessary Libraries

In [1]:
import json
from sklearn.metrics import cohen_kappa_score
import pandas as pd

Reading CSV files for Annotator1 and Annotator2

In [2]:
df_annotator_1 = pd.read_csv('Annotator1.csv')
df_annotator_2 = pd.read_csv('Annotator2.csv')

Function to extract words and their corresponding labels from the JSON format in 'label' column

In [3]:
def extract_words_and_labels(label_json):
    data = json.loads(label_json)
    return [(item["text"], item["labels"][0]) for item in data] 

Creating a list of final data

In [4]:
final_data = []
for _, row_1 in df_annotator_1.iterrows():
    annotation_id = row_1["annotation_id"]
    
    #extracting Annotator 1's words and labels
    words_and_labels_1 = extract_words_and_labels(row_1["label"])

    #finding the corresponding row in Annotator 2's data
    row_2 = df_annotator_2[df_annotator_2["annotation_id"] == annotation_id].iloc[0]

    #extracting Annotator 2's words and labels
    words_and_labels_2 = extract_words_and_labels(row_2["label"])

    #combining the words and annotations from both annotators
    for word_and_label_1, word_and_label_2 in zip(words_and_labels_1, words_and_labels_2):
        word_1, label_1 = word_and_label_1
        word_2, label_2 = word_and_label_2
        final_data.append([annotation_id, word_1, label_1, label_2])
    
    df_final = pd.DataFrame(final_data, columns=["annotation_id", "word", "annotator_1_label", "annotator_2_label"])

In [5]:
#printing the first 5 rows of final dataframe
print(df_final.head(5))

   annotation_id    word annotator_1_label annotator_2_label
0              1    कथित               ADJ               ADJ
1              1   Delhi             PROPN             PROPN
2              1  Liquor              NOUN              NOUN
3              1    Scam              NOUN              NOUN
4              1      पर               ADP               ADP


In [6]:
#printing the last 5 rows of final dataframe
print(df_final.tail(5))

     annotation_id  word annotator_1_label annotator_2_label
341             20    से               ADP               ADV
342             20  पूछे              VERB              VERB
343             20  तीखे               ADJ               ADJ
344             20  सवाल              NOUN              NOUN
345             20     !                 X                 X


Calculating Cohen's Kappa using the inbuilt function

In [7]:
annotator_1_labels = df_final['annotator_1_label']
annotator_2_labels = df_final['annotator_2_label']

kappa_score = cohen_kappa_score(annotator_1_labels, annotator_2_labels)

#printing the result
print(f"Cohen's Kappa: {kappa_score}")

Cohen's Kappa: 0.7756229038108845


### Interpretation for Cohen's Kappa

The agreement score for NER Task is 0.7756229038108845 which seems like a decent agreement score. However, still some improvements can be made. The score is almost close to 0.8 which shows that there is a fairly good agreement between the annotators. The potential reasons for disagreement could be due to differences in interpretation of annotations.

### Task 3.a Calculating Cohen's Kappa

Calculating Fleiss's Kappa using inbuilt function

Importing Neccessary Libraries

In [10]:
import pandas as pd
from statsmodels.stats.inter_rater import fleiss_kappa

Calculating Fleiss' Kappa

In [11]:
file_annotator1 = 'Annotator1CV.csv'  
file_annotator2 = 'Annotator2CV.csv'
file_annotator3 = 'Annotator3CV.csv'

#dataframes
df1 = pd.read_csv(file_annotator1)
df2 = pd.read_csv(file_annotator2)
df3 = pd.read_csv(file_annotator3)

df1 = df1[['annotation_id', 'choice']].rename(columns={'choice': 'Annotator1'})
df2 = df2[['annotation_id', 'choice']].rename(columns={'choice': 'Annotator2'})
df3 = df3[['annotation_id', 'choice']].rename(columns={'choice': 'Annotator3'})

#merging the dataframes on annotation_id
df_combined = df1.merge(df2, on='annotation_id').merge(df3, on='annotation_id')

#creating a frequency matrix
df_long = df_combined.melt(id_vars=['annotation_id'], value_vars=['Annotator1', 'Annotator2', 'Annotator3'],
                           var_name='annotator', value_name='choice')

#pivot to create a frequency matrix
matrix = df_long.pivot_table(index='annotation_id', columns='choice', aggfunc='size', fill_value=0)

#calculating Fleiss' Kappa
kappa = fleiss_kappa(matrix.to_numpy(), method='fleiss')

#printing the result
print(f"Fleiss' Kappa: {kappa}")

Fleiss' Kappa: 0.6662958843159064


### Interpretation for Fleiss' Kappa

The agreement score for Image Classification task is 0.6662958843159064. This shows a good enough agreement between the annotators. The discrepencies might be due to subjective interpretations.